<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/intel_images_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 19th October, 2021
# Ref: https://www.kaggle.com/puneet6060/intel-image-classification
#     https://www.kaggle.com/accountstatus/using-keras-cnn-on-intel-data
#
#  Objectives:
#            a. Build a simple CNN model for image classfication
#            b. Save the model, host it on github and use it for
#               developing a webapp.
#            c. webapp file is: imgclass_app.py
#            d. Images are stored on gdrive under:
#                 ..Colab_data_files/intel_images
#            e. And saved model (.h5 file) under:
#                 ..Colab_data_files/intelmodel


In [ ]:
# 0.0 We need to install h5py to save our model:

! pip install pyyaml h5py


In [ ]:
# 0.1 Display outputs of multiple commands from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Call libraries

In [ ]:
# 1.0 Call libraries
#import pandas as pd
import numpy as np

# 1.1
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import save_model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten,Conv2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# 1.2
from PIL import Image
import matplotlib.pyplot as plt
import os


## Design model

In [ ]:
# 2.0 Creating a simple sequential model

model=Sequential()
model.add(Convolution2D(
                        32,
                        (3,3),
                        input_shape=(64,64,3),
                        activation='relu'
                        )
           )

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D( 64,
                        (3,3),
                        activation='relu'
                        )
           )

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D( 128,
                        (3,3),
                        activation='relu'
                        )
           )

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(300,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=6,activation='softmax'))

In [ ]:
# 2.1 Get model summary
model.summary()

In [ ]:
# 2.2 Plot model:

tf.keras.utils.plot_model(
                           model,
                           to_file="model.png",
                           show_shapes=False,
                           show_layer_names=True,
                           rankdir="TB",
                           expand_nested=False
                          )

In [ ]:
# 2.3 Compile model:

model.compile(
              optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy']
              )

## Image augmentation

In [ ]:
# 3.0 Instantiate ImageDataGenerator:

train_datagen = ImageDataGenerator(
                                   rescale = 1./255, 
                                   shear_range = 0.2,    # Increases delay
                                   zoom_range = 0.2,     # increases delay
                                   horizontal_flip = True
                                   )

In [ ]:
# 3.1 Mount gdrive
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# 3.2 Where is our train and test data:

train_path_disk = '/gdrive/MyDrive/Colab_data_files/intel_images/seg_train' 
test_path_disk = '/gdrive/MyDrive/Colab_data_files/intel_images/seg_test'

In [ ]:
# 3.3 Transfer the train and test data to VM for
#     speeding up analysis:

! mkdir /content/train/
! mkdir /content/test/


In [ ]:
# 3.4 Now transfer zipped files
#     After transfer, modeling is very fast
%%time

! cp -R /gdrive/MyDrive/Colab_data_files/intel_images/seg_train.zip  /content/train/
! cp -R /gdrive/MyDrive/Colab_data_files/intel_images/seg_test.zip  /content/test/

In [ ]:
# 3.4.1 Check contents:
! ls /content/train
! ls /content/test

In [ ]:
# 3.4.2 Unzip files will be in the current folder 
#       Current folder is /content/
#       Check with !pwd 

! unzip /content/train/seg_train.zip
! unzip /content/test/seg_test.zip

In [ ]:
# 3.5 Train/test paths of data:

train_path = '/content/seg_train' 
test_path = '/content/seg_test'

In [ ]:
# 3.6 Create training data generator:

training_set = train_datagen.flow_from_directory(
                                                  train_path,
                                                  target_size = (64, 64), 
                                                  batch_size = 32
                                                 )


In [ ]:
# 3.7 This is our test data generator:

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
                                            test_path,
                                            target_size = (64, 64),
                                            batch_size = 32
                                            )

In [ ]:
# 3.8 How many classes are there:

len(set(training_set.classes))

# 3.9 And which ones:

l=os.listdir(train_path)
l.sort()
l          # ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

## Train model

In [ ]:
# 4.0 Delete folder(s) checkpoint
#     from /tmp where best model-wts
#     are stored:

! rm -r -f /tmp/checkpoint
! rm -r -f /tmp/checkpoint*

In [ ]:
# 4.1 Define a callback function
#     to store best possible weights:

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = ModelCheckpoint(
                                            filepath=checkpoint_filepath,
                                            save_weights_only=True,
                                            monitor='val_accuracy',
                                            mode='max',
                                            save_best_only=True
                                            )

In [ ]:
# 4.2 Fit the model:
fitted_model = model.fit(
                           training_set,
                           steps_per_epoch = 100,      # select 50, if short of time
                           epochs = 70,                # Select 5, if short of time
                           validation_data = test_set,
                           callbacks=[model_checkpoint_callback],
                           validation_steps = 20
                         )

## Save best weights as h5 file
Save best weights stored by checkpoint callbacks to disk


In [ ]:
# 5.1 Folder where trained model will be saved in gdrive:

! mkdir /gdrive/MyDrive/Colab_data_files/intelmodel

In [ ]:
# 5.2 This will be our model path and file:

pathToSave = "/gdrive/MyDrive/Colab_data_files/intelmodel/"

In [ ]:
# 5.3 The model weights (that are considered 
#      the best) are loaded into the model.
#       (But model must exist aprior):

model.load_weights(checkpoint_filepath)

In [ ]:
# 5.4 Now that model is loaded with best weights, 
#     we save the complete model. It will be 
#     saved to pwd (present working directory)
#     ie /content

model.save("intel_images_model.h5")

In [ ]:
# 5.5 Transfer it to gdrive:
! rm /gdrive/MyDrive/Colab_data_files/intelmodel/intel_images_model.h5 
! cp /content/intel_images_model.h5  /gdrive/MyDrive/Colab_data_files/intelmodel/

In [ ]:
# 5.6  Load the saved model:

pathToSave = "/gdrive/MyDrive/Colab_data_files/intelmodel/"
mymodel = keras.models.load_model(pathToSave+"intel_images_model.h5")

## Save model directly

In [ ]:
# 6.0 Else, forget checkpointing, save the 'last' complete model 
#      as it is:

#model.save(pathToSave + 'intel_images_model.h5')

In [ ]:
# 6.1  Load the saved model:

#pathToSave = "/gdrive/MyDrive/Colab_data_files/intelmodel/"
#mymodel = keras.models.load_model(pathToSave+"intel_images_model.h5")

In [ ]:
# 6.1.1:

! rm -r /content/saved_model

In [ ]:
# 6.3 Create a folder in pwd (/content)
#      Save the entire model within it.
#       Model folder is: my_model

!mkdir -p /content/saved_model

In [ ]:
# 6.4

model.save('saved_model/my_model')

In [ ]:
# 6.5 Delete any existing model from gdrive, if it exists:

! rm -r /gdrive/MyDrive/Colab_data_files/intelmodel/saved_model

In [ ]:
# 6.6 Check:

! ls -la /gdrive/MyDrive/Colab_data_files/intelmodel/

In [ ]:
# 6.7 Next copy the saved model from VM to gdrive:
#     First create the folder, if it does not exist:

! mkdir /gdrive/MyDrive/Colab_data_files/intelmodel

In [ ]:
# 6.8 Now copy it. Wait for some time
#     as copying to gdrive takes time

! cp -r /content/saved_model  /gdrive/MyDrive/Colab_data_files/intelmodel/

In [ ]:
# 6.9 Check again:
! ls -la /gdrive/MyDrive/Colab_data_files/intelmodel/

In [ ]:
# 6.10 Get size of saved_model folder:

! du /gdrive/MyDrive/Colab_data_files/intelmodel/saved_model  

In [ ]:
# 6.11 Get summary size
! du /gdrive/MyDrive/Colab_data_files/intelmodel/saved_model  -sh

In [ ]:
# 6.12 You can load this saved model as:

new_model = keras.models.load_model('/content/saved_model/my_model')
mymodel = new_model

## Save best weights model as a folder
Not as a h5 file. Good for Heroku 

In [ ]:
# 7.0 Delete earlier saved model, if any:

! rm -r -f /content/saved_model

In [ ]:
#  7.1 The model weights (that are considered 
#       the best) are loaded into the model.
#        (But model must exist aprior):

model.load_weights(checkpoint_filepath)

In [ ]:
# 7.2 Create a folder in pwd (/content)
#      Save the entire model within it.
#       Model folder is: my_model

!mkdir -p /content/saved_model

In [ ]:
# 7.2.1

model.save('saved_model/my_model')

In [ ]:
# 7.3 Delete existing model from gdrive, if it exists:

! rm -r /gdrive/MyDrive/Colab_data_files/intelmodel/saved_model

In [ ]:
# 7.4 Next copy the saved model from VM to gdrive:
#     First create the folder, if it does not exist:

! mkdir /gdrive/MyDrive/Colab_data_files/intelmodel


In [ ]:
# 7.4.1 Now save it:

! cp -r /content/saved_model  /gdrive/MyDrive/Colab_data_files/intelmodel/

In [ ]:
# 7.5 You can load this saved model as:

new_model = keras.models.load_model('/content/saved_model/my_model')
mymodel = new_model

## Get model size

In [ ]:
# 7.0 Get size of h5 file:

!ls -la /gdrive/MyDrive/Colab_data_files/intelmodel/intel_images_model.h5

In [ ]:
# 7.1 Get size of saved_model folder:

! du /gdrive/MyDrive/Colab_data_files/intelmodel/saved_model  -sh

## Load model & experiment

In [ ]:
# 8.0 Delete existing model:

del model

In [ ]:
# 8.1 Here are few sample images 
#     to be tested:

image_path_forest = "/content/seg_train/forest/10007.jpg"
image_path_building = "/content/seg_train/buildings/10006.jpg"
image_path_glacier = "/content/seg_train/glacier/10.jpg"
image_path_mountain = "/content/seg_train/mountain/10008.jpg"
image_path_sea = "/content/seg_train/sea/10016.jpg"
image_path_street = "/content/seg_train/street/10015.jpg"

In [ ]:
# 8.2 Define a prediction function:

def predict(image_path): 
    model = mymodel
    image = load_img(image_path, target_size=(64, 64))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #image = preprocess_input(image)
    yhat = model.predict(image)
    classes_x=np.argmax(yhat,axis=1)
    l=os.listdir(train_path)
    l.sort()
    return l[classes_x[0]]

In [ ]:
# 7.3 Make predictions:

print(predict(image_path_street))
print(predict(image_path_forest))
print(predict(image_path_sea))
print(predict(image_path_building))
print(predict(image_path_mountain))

In [ ]:
####### I am done ##############